# Does the poker code run?

In [1]:
import numpy as np
import itertools
from collections import namedtuple
from poker import Card, Evaluator, Player, Game

In [2]:
g = Game(6, 100, 10000) 

In [6]:
g.play_one_complete_round()

The position of the small blind is 2
[1, 2]
[0, 1, 2, 3, 4, 5]
[0, 3, 4, 5]
[3, 4, 5] [0]
[1, 2, 3, 4, 5, 0]
The order is [1, 2, 3, 4, 5, 0]
1's hand is [('8', '♣'), ('q', '♥')] 
2's hand is [('7', '♠'), ('8', '♦')] 
3's hand is [('5', '♠'), ('9', '♥')] 
4's hand is [('k', '♦'), ('10', '♠')] 
5's hand is [('5', '♦'), ('2', '♦')] 
6's hand is [('3', '♦'), ('7', '♦')] 
Player 2: I have bet the small blind of $100!
Player 3: I have bet the big blind of $200!
Player 1: I think I can win this round. I am calling!
Player 1: I have call and am betting $200!
Player 4: I think I can win this round. I am calling!
Player 4: I have call and am betting $200!
Player 5: I think I can win this round. I am calling!
Player 5: I have call and am betting $200!
Player 6: I have raise and am betting $400!
Player 2: I think I can win this round. I am calling!
Player 2: I have call and am betting $400!
The highest bid is 400
Player 3: My cards are sh..! I fold!
Player 3: I have fold and am betting $200!
The h